# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [4]:
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.organization_cyber_question import OrganizationCyberModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd


In [ ]:
sqlstorage = SQLiteStorageProvider("organization_cyber.db")

workflow = Workflow(
    SonarQueryHandler(OrganizationCyberModel),
    sqlstorage,
    workers = 4,
    )

data = pd.read_excel("organization_names.xlsx")
organizations = data["organization_name"].tolist()
countries = data["country"].tolist()

question_set = get_question_set(
    organizations=organizations,
    countries=countries,
)

question_set.max_questions = 10  # No limit

Loaded 0 stored question hashes from organization_cyber.db


In [ ]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 193 questions with 4 workers
Processing question 1/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Public Works, Transportation and Communications (MTPTC) in HAITI'}
Processing question 2/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Mines and Energy in CAMBODIA'}
Processing question 3/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Electric Power and Petroleum (Ministerio del Poder Popular de Energía y Petróleo - MPPEP) in VENEZUELA'}
Processing question 4/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Energy and Mines (Ministerio de Energía y Minas) in CUBA'}
Finished question 4/193
Processing question 5/193: Assess the cybersecurity responsibility of {organiza

ValidationError: 1 validation error for OrganizationCyberModel
responsible_date
  Input should be a valid date or datetime, invalid character in year [type=date_from_datetime_parsing, input_value='ongoing (no evidence for a specific start date)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/date_from_datetime_parsing

Finished question 16/193
Processing question 17/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Economy and Territories (Ministeri d’Economia I Territori) in ANDORRA'}
Finished question 17/193
Processing question 18/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Ecological Transition (Ministère de la Transition écologique) in FRANCE'}
Finished question 18/193
Processing question 19/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': 'Ministry of Mines and Energy in NAMIBIA'}
Finished question 19/193
Processing question 20/193: Assess the cybersecurity responsibility of {organization_country}.  - {'organization_country': "Ministry of Energy, Mines and Quarries (Ministère de l'Énergie, des Mines et des Carrières) in BURKINA FASO"}
Finished question 20/193
Processing question 21/193: Assess the cybersecurity resp

In [ ]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened

,question,error,organization_country,organization,responsible_date,responsibility_level,explanation,confidence,enriched_citations
0,Assess the cybersecurity responsibility of `Mi...,None,`Ministry of Energy and Renewable Energy` in A...,Ministry of Energy and Renewable Energy,None,ResponsibilityLevel.LOW,The Ministry of Energy and Renewable Energy in...,ConfidenceLevel.HIGH,[{'url': 'https://www.energycentral.com/renewa...
1,Assess the cybersecurity responsibility of `Mi...,None,`Ministry of Energy and Renewable Energy` in A...,Ministry of Energy and Renewable Energy,None,ResponsibilityLevel.LOW,There is no evidence among the provided source...,ConfidenceLevel.MEDIUM,[{'url': 'https://www.ere.gov.al/media/files/2...
2,Assess the cybersecurity responsibility of `Mi...,None,`Ministry of Energy and Renewable Energy` in A...,Ministry of Energy and Renewable Energy,None,ResponsibilityLevel.LOW,The Ministry of Energy and Renewable Energy in...,ConfidenceLevel.MEDIUM,[{'url': 'https://www.trade.gov/country-commer...
3,Assess the cybersecurity responsibility of `Mi...,None,`Ministry of Energy and Renewable Energy` in A...,Ministry of Energy and Renewable Energy,None,ResponsibilityLevel.NONE,No evidence was found in the search results to...,ConfidenceLevel.MEDIUM,[{'url': 'https://www.techmonitor.ai/cybersecu...


In [ ]:
wrangled = flattened.copy()


In [ ]:
wrangled.enriched_citations

0    [{'url': 'https://www.energycentral.com/renewa...
1    [{'url': 'https://www.ere.gov.al/media/files/2...
2    [{'url': 'https://www.trade.gov/country-commer...
3    [{'url': 'https://www.techmonitor.ai/cybersecu...
Name: enriched_citations, dtype: object

In [ ]:
wrangled.to_excel("organization_cyber.xlsx", index=False)